In [ ]:
import os
project_dir = "/Volumes/mnt/Workspace/ai-trading-agents"
os.chdir(project_dir)

In [ ]:
get_ipython().run_line_magic('reload_ext', 'autoreload')  # type: ignore
get_ipython().run_line_magic('autoreload', '2')

In [ ]:
from src import secrets
from src.models import get_model
from langchain.agents import create_agent
from langchain_core.messages import HumanMessage, AIMessage, ToolMessage

In [ ]:
SECRETS = secrets.load()
ENV = os.environ.get("ENV", "dev")

In [ ]:
instruction = "Please recommend a way how to treat the depresssion of motion by reasoning the user's problem."

In [ ]:
start_messages = [HumanMessage(content=instruction)]

In [ ]:
langchain_model = get_model("minimax/minimax-m2.1")

In [ ]:
agent = create_agent(
    model=langchain_model
)

In [ ]:
response = agent.invoke({"messages": start_messages})

In [ ]:
deserialized_messages = []

In [ ]:
for message in response['messages']:
    deserialized_messages.append(message.model_dump())

In [ ]:
deserialized_messages_with_reasoning = deserialized_messages.copy()

In [ ]:
deserialized_messages_without_reasoning = deserialized_messages.copy()

In [ ]:
for message in deserialized_messages_without_reasoning:
    if message['type'] == 'ai':
        del message['additional_kwargs']['reasoning_content']

In [ ]:
message_type_map = {"ai": AIMessage,
                    "human": HumanMessage, "tool": ToolMessage}

In [ ]:
serialized_messages_with_reasoning = [
    message_type_map[msg["type"]](**msg) for msg in deserialized_messages_with_reasoning
]

In [ ]:
serialized_messages_without_reasoning = [
    message_type_map[msg["type"]](**msg) for msg in deserialized_messages_without_reasoning
]

In [ ]:
serialized_messages_with_reasoning.append(HumanMessage(content="Thank you"))

In [ ]:
reasoning_response = agent.invoke(
    {"messages": serialized_messages_with_reasoning})

In [ ]:
reasoning_response['messages'][-1].usage_metadata

In [ ]:
serialized_messages_without_reasoning.append(HumanMessage(content="Thank you"))

In [ ]:
without_reasoning_response = agent.invoke(
    {"messages": serialized_messages_without_reasoning})

In [ ]:
without_reasoning_response['messages'][-1].usage_metadata

In [ ]:
without_reasoning_response['messages'][-1].content = "replaced"

In [ ]:
without_reasoning_response['messages'][-1].content